# Utils and installations

In [16]:
def open_file(path_to_txt):
    f = open(path_to_txt, 'r')
    txt = [el for el in f.read().split('\n') if el != '']
    f.close()
    return txt


def write_file(path_save, to_write):
    f = open(path_save, 'w')
    f.truncate(0)
    if (to_write[-1] == '\n'):
        f.write(to_write[:-1])
    else:
        f.write(to_write)
    f.close()

In [ ]:
!sudo apt-get install festival espeak-ng mbrola

In [ ]:
!pip install phonemizer

In [ ]:
!pip install transformers

# Quantized for word

In [20]:
def get_quantized_word(paths_to_task, type_task, path_save_quantized):
    
    if type_task not in ['similarity_task', 'score_task']:
        raise ValueError('type_task should be similarity_task or score_task.')
    
    task = [el for path in paths_to_task for el in open_file(path)]
    dic_code_sent = {}
    
    if type_task == 'similarity_task':
        for test in task:
            for word in test.split(' '):
                dic_code_sent[word] = word
                
    elif type_task == 'score_task':
        for test in task:
            code_1, code_2, sent_1, sent_2 = test.split(',')
            dic_code_sent[code_1] = sent_1
            dic_code_sent[code_2] = sent_2
            
    to_write = ['\t'.join([code, dic_code_sent[code]]) for code in dic_code_sent]
    to_write = '\n'.join(to_write)
    write_file(path_save_quantized, to_write)


In [ ]:
# paths_to_task = ['/Data/tasks_quantized_scores/similarity_tasks/syn/task/syn_dev.txt',
#                  '/Data/tasks_quantized_scores/similarity_tasks/syn/task/syn_test.txt']
# type_task = 'similarity_task'
# path_save_quantized = '/Data/tasks_quantized_scores/similarity_tasks/syn/quantized/word/quantized.txt'
# get_quantized_word(paths_to_task, type_task, path_save_quantized)


# paths_to_task = ['/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_dev.txt',
#                  '/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_test.txt']
# type_task = 'score_task'
# path_save_quantized = '/Data/tasks_quantized_scores/score_tasks/aga_easy/quantized/word/quantized.txt'
# get_quantized_word(paths_to_task, type_task, path_save_quantized)

# Quantized for phone nobound and phone bound

In [21]:
import os
import pickle


def get_quantized_phone_nobound_bound(paths_to_task,
                                      type_task,
                                      path_dict_encode_phonemes,
                                      path_save_quantized,
                                      phone_nobound):
    
    if type_task not in ['similarity_task', 'score_task']:
        raise ValueError('type_task should be similarity_task or score_task.')

    get_quantized_word(paths_to_task, type_task, path_save_quantized)
    quantized_word = open_file(path_save_quantized)

    codes = [el.split('\t')[0] for el in quantized_word]
    sents = [el.split('\t')[1] for el in quantized_word]

    write_file(path_save_quantized, '\n'.join(sents))

    os.system(f"phonemize -b espeak -l en-us -p '-' -w ' ' '{path_save_quantized}' -o '{path_save_quantized + '.txt'}'")
    os.rename(path_save_quantized + '.txt', path_save_quantized)

    sents_phone = open_file(path_save_quantized)
    sents_phone = [el[:-1].split(' ') for el in sents_phone]

    file = open(path_dict_encode_phonemes, "rb")
    dict_encode_phonemes = pickle.load(file)
    file.close()

    sep = ','
    if not phone_nobound:
        sep = ',0,'

    quantized_phone = []
    for i, sent_phone in enumerate(sents_phone):
        try:
            sent_phone_encoded = [','.join(list(map(dict_encode_phonemes.get, word.split('-')[:-1]))) for word in sent_phone]
            quantized_phone.append('\t'.join([codes[i], sep.join(sent_phone_encoded)]))
        except:
            pass

    write_file(path_save_quantized, '\n'.join(quantized_phone))


In [ ]:
# paths_to_task = ['/Data/tasks_quantized_scores/similarity_tasks/syn/task/syn_dev.txt',
#                  '/Data/tasks_quantized_scores/similarity_tasks/syn/task/syn_test.txt']
# type_task = 'similarity_task'
# path_dict_encode_phonemes = '/Data/models/topline_phoneme_smallbert/construction/dict_encode_phonemes.pkl'
# path_save_quantized = '/Data/tasks_quantized_scores/similarity_tasks/syn/quantized/phoneme_no_boundaries/quantized.txt'
# phone_nobound = True
# get_quantized_phone_nobound_bound(paths_to_task, type_task, path_dict_encode_phonemes, path_save_quantized, phone_nobound)


# paths_to_task = ['/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_dev.txt',
#                  '/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_test.txt']
# type_task = 'score_task'
# path_dict_encode_phonemes = '/Data/models/topline_phoneme_smallbert/construction/dict_encode_phonemes.pkl'
# path_save_quantized = '/Data/tasks_quantized_scores/score_tasks/aga_easy/quantized/phoneme_no_boundaries/quantized.txt'
# phone_nobound = True
# get_quantized_phone_nobound_bound(paths_to_task, type_task, path_dict_encode_phonemes, path_save_quantized, phone_nobound)

# Quantized for phone nobound masking

In [22]:
def get_quantized_phone_nobound_masking(paths_to_task,
                                        type_task,
                                        path_dict_encode_phonemes,
                                        path_save_quantized):
  
    if type_task not in ['similarity_task', 'score_task']:
        raise ValueError('type_task should be similarity_task or score_task.')
  
    get_quantized_phone_nobound_bound(paths_to_task,
                                      type_task,
                                      path_dict_encode_phonemes,
                                      path_save_quantized,
                                      phone_nobound = False)
    
    quantized_phone_bound = open_file(path_save_quantized)
    quantized_phone_bound = [el.split('\t') for el in quantized_phone_bound]

    for i in range(len(quantized_phone_bound)):
        list_int_pre = list(map(int, quantized_phone_bound[i][1].split(',')))

        list_int_post = list_int_pre * 1
        for k in range(1, len(list_int_post)):
            if (list_int_post[k-1] != 0) and (list_int_post[k] != 0):
                list_int_post[k] += 59
        list_int_post = [el for el in list_int_post if el != 0]

        list_string_post = list(map(str, list_int_post))
        quantized_phone_bound[i][1] = ','.join(list_string_post)

    quantized_phone_nobound_masking = ['\t'.join(el) for el in quantized_phone_bound]
    write_file(path_save_quantized, '\n'.join(quantized_phone_nobound_masking))


In [ ]:
# paths_to_task = ['/Data/tasks_quantized_scores/similarity_tasks/syn/task/syn_dev.txt',
#                  '/Data/tasks_quantized_scores/similarity_tasks/syn/task/syn_test.txt']
# type_task = 'similarity_task'
# path_dict_encode_phonemes = '/Data/models/topline_phoneme_smallbert/construction/dict_encode_phonemes.pkl'
# path_save_quantized = '/Data/tasks_quantized_scores/similarity_tasks/syn/quantized/phoneme_no_boundaries_masking/quantized.txt'
# get_quantized_phone_nobound_masking(paths_to_task, type_task, path_dict_encode_phonemes, path_save_quantized)


# paths_to_task = ['/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_dev.txt',
#                  '/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_test.txt']
# type_task = 'score_task'
# path_dict_encode_phonemes = '/Data/models/topline_phoneme_smallbert/construction/dict_encode_phonemes.pkl'
# path_save_quantized = '/Data/tasks_quantized_scores/score_tasks/aga_easy/quantized/phoneme_no_boundaries_masking/quantized.txt'
# get_quantized_phone_nobound_masking(paths_to_task, type_task, path_dict_encode_phonemes, path_save_quantized)

# Quantized for phone nobound masking bpe and phone nobound masking bpe nobound

## Train a BPE tokenizer

In [23]:
import string
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer


def encode_phoneme_bound(path_to_phoneme_bound, path_save_phonemes_encoded):
    
    f = open(path_to_phoneme_bound, 'r')
    phonemes = [list(map(int, el.split(' '))) for el in f.read().split('\n') if el != '']
    f.close()
    
    encoding_values = string.printable[:59]
    dic_encoder = {i: encoding_values[i-1] for i in range(1, 60)}
    dic_encoder[0] = ' '
    
    phonemes_encoded = [list(map(dic_encoder.get, el)) for el in phonemes]
    phonemes_encoded = [''.join(el) for el in phonemes_encoded]
    
    write_file(path_save_phonemes_encoded, '\n'.join(phonemes_encoded))
    
    return phonemes_encoded


def train_tokenizer(path_to_training_data, vocab_size, path_save_tokenizer):
    
    tokenizer = Tokenizer(BPE(unk_token = "<unk>"))
    tokenizer.pre_tokenizer = Whitespace()
    
    trainer = BpeTrainer(vocab_size = vocab_size, special_tokens = ["<unk>", "<s>", "</s>", "<pad>", "<mask>"])
    tokenizer.train(files = path_to_training_data, trainer = trainer)
    
    tokenizer.save(path_save_tokenizer)
    

def encode_with_tokenizer(path_to_saved_tokenizer, path_to_phonemes_encoded, path_save_final_encoding):
    
    def split_word(word):
        return [char for char in word]
    
    def add_59(list_):
        return [el + 59 if i > 0 else el for i, el in enumerate(list_)]

    f = open(path_to_phonemes_encoded, 'r')
    phonemes_encoded = f.read().split('\n')
    f.close()
    
    tokenizer = Tokenizer.from_file(path_to_saved_tokenizer)
    
    encoding_values = string.printable[:59]
    dic_decoder = {encoding_values[i-1]: i for i in range(1, 60)}
    
    to_write = ''
    for sent in phonemes_encoded:
        output_tokenizer = tokenizer.encode(sent)
        decoding_values = [list(map(dic_decoder.get, split_word(word))) for word in output_tokenizer.tokens]
        decoding_values = [add_59(el) for el in decoding_values]
        decoding_values = [str(el) for sublist in decoding_values for el in sublist]
        to_write += ' '.join(decoding_values) + '\n'
    
    write_file(path_save_final_encoding, to_write)
    

In [ ]:
# path_to_phoneme_bound = '/Data/models/topline_phoneme_smallbert/with_boundaries/construction/quantized/phonemes_withboundaries_preprocessed_train_text_librispeech.txt'
# path_save_phonemes_encoded = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/phonemes_before_bpe_encoding_train_text_librispeech.txt'
# phonemes = encode_phoneme_bound(path_to_phoneme_bound, path_save_phonemes_encoded)



# path_to_training_data = ['/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/phonemes_before_bpe_encoding_train_text_librispeech.txt',
#                          '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/phonemes_before_bpe_encoding_dev_text_librispeech.txt',
#                          '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/phonemes_before_bpe_encoding_test_text_librispeech.txt']
# vocab_size = 1000
# path_save_tokenizer = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/tokenizer_bpe_1000.json'
# train_tokenizer(path_to_training_data, vocab_size, path_save_tokenizer)


# path_to_training_data = ['/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/construction/phonemes_before_bpe_encoding_train_text_librispeech.txt',
#                          '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/construction/phonemes_before_bpe_encoding_dev_text_librispeech.txt',
#                          '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/construction/phonemes_before_bpe_encoding_test_text_librispeech.txt']
# vocab_size = 1000
# path_save_tokenizer = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/construction/tokenizer_bpe_1000.json'
# train_tokenizer(path_to_training_data, vocab_size, path_save_tokenizer)



# path_to_saved_tokenizer = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/tokenizer_bpe_1000.json'
# path_to_phonemes_encoded = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/phonemes_before_bpe_encoding_train_text_librispeech.txt'
# path_save_final_encoding = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/no_boundaries_masking_bpe_1000/phonemes_noboundaries_masking_bpe_1000_preprocessed_train_text_librispeech.txt'
# encode_with_tokenizer(path_to_saved_tokenizer, path_to_phonemes_encoded, path_save_final_encoding)


# path_to_saved_tokenizer = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/construction/tokenizer_bpe_1000.json'
# path_to_phonemes_encoded = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/construction/phonemes_before_bpe_encoding_train_text_librispeech.txt'
# path_save_final_encoding = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/no_boundaries_masking_bpe_nobound_1000/phonemes_noboundaries_masking_bpe_nobound_1000_preprocessed_train_text_librispeech.txt'
# encode_with_tokenizer(path_to_saved_tokenizer, path_to_phonemes_encoded, path_save_final_encoding)


## Create quantized

In [24]:
def get_quantized_phone_nobound_masking_bpe(paths_to_task,
                                            type_task,
                                            path_dict_encode_phonemes,
                                            path_to_tokenizer,
                                            path_to_save_quantized,
                                            nobound_bpe_nobound):
  
    if type_task not in ['similarity_task', 'score_task']:
        raise ValueError('type_task should be similarity_task or score_task.')

    get_quantized_phone_nobound_bound(paths_to_task,
                                      type_task,
                                      path_dict_encode_phonemes,
                                      path_save_quantized,
                                      phone_nobound = False)
    
    f = open(path_to_save_quantized, 'r')
    task_phoneme_bound = [el.split('\t') for el in f.read().split('\n') if el != '']
    f.close()
    
    codes = [el[0] for el in task_phoneme_bound]
    phoneme_bound = [' '.join(el[1].split(',')) for el in task_phoneme_bound]
    if nobound_bpe_nobound:
        phoneme_bound = [' '.join(el[1].split(',')).replace(' 0 ', ' ') for el in task_phoneme_bound]
    
    write_file(path_to_save_quantized, '\n'.join(phoneme_bound))
    
    encode_phoneme_bound(path_to_save_quantized, path_to_save_quantized)
    
    encode_with_tokenizer(path_to_tokenizer, path_to_save_quantized, path_to_save_quantized)
    
    f = open(path_to_save_quantized, 'r')
    phoneme_nobound_bpe = [','.join(el.split(' ')) for el in f.read().split('\n') if el != '']
    f.close()
    
    assert len(codes) == len(phoneme_nobound_bpe)
    
    quantized = [codes[i] + '\t' + phoneme_nobound_bpe[i] for i in range(len(codes))]
    
    write_file(path_to_save_quantized, '\n'.join(quantized))

In [ ]:
# paths_to_task = ['/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_dev.txt',
#                  '/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_test.txt']
# type_task = 'score_task'
# path_dict_encode_phonemes = '/Data/models/topline_phoneme_smallbert/construction/dict_encode_phonemes.pkl'
# path_to_tokenizer = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/tokenizer_bpe_1000.json'
# path_save_quantized = '/Data/tasks_quantized_scores/score_tasks/aga_easy/quantized/phoneme_no_boundaries_masking_bpe_1000/quantized.txt'
# nobound_bpe_nobound = False
# get_quantized_phone_nobound_masking_bpe(paths_to_task, type_task, path_dict_encode_phonemes, path_to_tokenizer, path_save_quantized, nobound_bpe_nobound = False)


# Quantized for phone nobound onehot bpe and phone nobound onehot bpe nobound

## Create encoding onehot

In [25]:
import json


def create_encoding_onehot(path_to_masking_bpe_files, path_to_save_dic_encode_onehot):
    
    full_txt_bpe_masking = ''
    
    for path in path_to_masking_bpe_files:
        f = open(path, 'r')
        full_txt_bpe_masking += f.read() + '\n'
        f.close()
    
    full_txt_bpe_masking = [el for el in full_txt_bpe_masking.split('\n') if el != '']

    bpe_to_encode = set()
    for i in range(1, 60):
        bpe_to_encode.add(str(i))
    
    for line in full_txt_bpe_masking:
        line_split = line.split(' ')
        line_split = list(map(int, line_split))
        
        ind_beg_bpe = [i for i, el in enumerate(line_split) if el <= 59]
        
        for i in range(len(ind_beg_bpe) - 1):
            bound = ind_beg_bpe[i + 1] - ind_beg_bpe[i]
            new_bpe = line_split[ind_beg_bpe[i] : ind_beg_bpe[i] + bound]
            new_bpe = ' '.join(list(map(str, new_bpe)))
            bpe_to_encode.add(new_bpe)
        new_bpe = line_split[ind_beg_bpe[len(ind_beg_bpe) - 1]:]
        new_bpe = ' '.join(list(map(str, new_bpe)))
        bpe_to_encode.add(new_bpe)
            
    for i in range(1, 60):
        bpe_to_encode.remove(str(i))
        
    bpe_to_encode = {el: i+1  for i, el in enumerate([str(i) for i in range(1, 60)] + list(bpe_to_encode))}
    print("Length of the encoding dictionary:", len(bpe_to_encode))
    
    with open(path_to_save_dic_encode_onehot, 'w') as fp:
        json.dump(bpe_to_encode, fp)

    
def masking_bpe_to_onehot_bpe(path_to_masking_bpe, path_to_dic_encode_onehot_bpe, path_to_save_onehot_bpe):
    
    masking_bpe = open_file(path_to_masking_bpe)
    
    with open(path_to_dic_encode_onehot_bpe) as f:
        dic_encode_onehot_bpe = json.load(f)
        
    onehot_bpe = ''
    
    for line in masking_bpe:
        line_split = line.split(' ')
        line_split = list(map(int, line_split))
        
        ind_beg_bpe = [i for i, el in enumerate(line_split) if el <= 59]
        
        new_line = []
        for i in range(len(ind_beg_bpe) - 1):
            bound = ind_beg_bpe[i + 1] - ind_beg_bpe[i]
            new_bpe = line_split[ind_beg_bpe[i] : ind_beg_bpe[i] + bound]
            new_bpe = ' '.join(list(map(str, new_bpe)))
            new_line.append(str(dic_encode_onehot_bpe[new_bpe]))
        new_bpe = line_split[ind_beg_bpe[len(ind_beg_bpe) - 1]:]
        new_bpe = ' '.join(list(map(str, new_bpe)))
        new_line.append(str(dic_encode_onehot_bpe[new_bpe]))
        onehot_bpe += ' '.join(new_line) + '\n'
        
    write_file(path_to_save_onehot_bpe, onehot_bpe)


In [ ]:
# path_to_masking_bpe_files = ['/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/no_boundaries_masking_bpe_1000/phonemes_noboundaries_masking_bpe_1000_preprocessed_train_text_librispeech.txt',
#                              '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/no_boundaries_masking_bpe_1000/phonemes_noboundaries_masking_bpe_1000_preprocessed_dev_text_librispeech.txt',
#                              '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/no_boundaries_masking_bpe_1000/phonemes_noboundaries_masking_bpe_1000_preprocessed_test_text_librispeech.txt']
# path_to_save_dic_encode_onehot = '/Data/models/topline_phoneme_smallbert/no_boundaries_onehot_bpe/construction/dic_encode_onehot_bpe_1000.json'
# create_encoding_onehot(path_to_masking_bpe_files, path_to_save_dic_encode_onehot)


# path_to_masking_bpe = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/no_boundaries_masking_bpe_1000/phonemes_noboundaries_masking_bpe_1000_preprocessed_train_text_librispeech.txt'
# path_to_dic_encode_onehot_bpe = '/Data/models/topline_phoneme_smallbert/no_boundaries_onehot_bpe/construction/dic_encode_onehot_bpe_1000.json'
# path_to_save_onehot_bpe = '/Data/models/topline_phoneme_smallbert/no_boundaries_onehot_bpe/no_boundaries_onehot_bpe_1000/construction/quantized/phonemes_noboundaries_onehot_bpe_1000_preprocessed_train_text_librispeech.txt'
# masking_bpe_to_onehot_bpe(path_to_masking_bpe, path_to_dic_encode_onehot_bpe, path_to_save_onehot_bpe)
        

## Create quantized

In [26]:
def get_quantized_phone_nobound_onehot_bpe(paths_to_task,
                                           type_task,
                                           path_dict_encode_phonemes,
                                           path_to_tokenizer,
                                           path_to_dic_encode_onehot_bpe,
                                           path_to_save_quantized,
                                           nobound_bpe_nobound):
  
    if type_task not in ['similarity_task', 'score_task']:
        raise ValueError('type_task should be similarity_task or score_task.')
    
    get_quantized_phone_nobound_masking_bpe(paths_to_task,
                                            type_task,
                                            path_dict_encode_phonemes,
                                            path_to_tokenizer,
                                            path_to_save_quantized,
                                            nobound_bpe_nobound)

    masking_bpe = open_file(path_to_save_quantized)
    masking_bpe = [el.split('\t') for el in masking_bpe]
    codes_quantized = [el[0] for el in masking_bpe]
    masking_bpe = [el[1] for el in masking_bpe]
    
    masking_bpe = '\n'.join(masking_bpe)
    masking_bpe = masking_bpe.replace(',', ' ')

    write_file(path_to_save_quantized, masking_bpe)
    
    masking_bpe_to_onehot_bpe(path_to_save_quantized, path_to_dic_encode_onehot_bpe, path_to_save_quantized)
    
    f = open(path_to_save_quantized, 'r')
    onehot_bpe = f.read()
    f.close()
    
    onehot_bpe = onehot_bpe.replace(' ', ',')
    onehot_bpe = onehot_bpe.split('\n')
    onehot_bpe = ['\t'.join([codes_quantized[i], el]) for i, el in enumerate(onehot_bpe)]
    onehot_bpe = '\n'.join(onehot_bpe)

    write_file(path_to_save_quantized, onehot_bpe)


In [ ]:
# paths_to_task = ['/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_dev.txt',
#                  '/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_test.txt']
# type_task = 'score_task'
# path_dict_encode_phonemes = '/Data/models/topline_phoneme_smallbert/construction/dict_encode_phonemes.pkl'
# path_to_tokenizer = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/construction/tokenizer_bpe_1000.json'
# path_to_dic_encode_onehot_bpe = '/Data/models/topline_phoneme_smallbert/no_boundaries_onehot_bpe/construction/dic_encode_onehot_bpe_1000.json'
# path_to_save_quantized = '/Data/tasks_quantized_scores/score_tasks/aga_easy/quantized/phoneme_no_boundaries_onehot_bpe_1000/quantized.txt'
# nobound_bpe_nobound = False
# get_quantized_phone_nobound_onehot_bpe(paths_to_task, type_task, path_dict_encode_phonemes, path_to_tokenizer, path_to_dic_encode_onehot_bpe, path_to_save_quantized, nobound_bpe_nobound)


# paths_to_task = ['/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_dev.txt',
#                  '/Data/tasks_quantized_scores/score_tasks/aga_easy/task/aga_easy_test.txt']
# type_task = 'score_task'
# path_dict_encode_phonemes = '/Data/models/topline_phoneme_smallbert/construction/dict_encode_phonemes.pkl'
# path_to_tokenizer = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe_nobound/construction/tokenizer_bpe_1000.json'
# path_to_dic_encode_onehot_bpe = '/Data/models/topline_phoneme_smallbert/no_boundaries_onehot_bpe_nobound/construction/dic_encode_onehot_bpe_nobound_1000.json'
# path_to_save_quantized = '/Data/tasks_quantized_scores/score_tasks/aga_easy/quantized/phoneme_no_boundaries_onehot_bpe_nobound_1000/quantized.txt'
# nobound_bpe_nobound = True
# get_quantized_phone_nobound_onehot_bpe(paths_to_task, type_task, path_dict_encode_phonemes, path_to_tokenizer, path_to_dic_encode_onehot_bpe, path_to_save_quantized, nobound_bpe_nobound)


# Create quantized for phone onehot dpparse and phone onehot dpparse noduration

## Create encoding dpparse

In [ ]:
def create_dic_encode_phonemes(path_to_segmentation, path_to_save_dic_encode):
    
    segm = open_file(path_to_segmentation)
    segm = [el.split(' ') for el in segm]
    segm = [el_ for el in segm for el_ in el]
    segm = set(segm)
    
    dic_encode = {el: str(i+1) for i, el in enumerate(segm)}
    print("Length of the encoding dictionary:", len(dic_encode))
    
    with open(path_to_save_dic_encode, 'w') as fp:
        json.dump(dic_encode, fp)
        

def create_quantized_libri(path_to_segmentation, path_to_dic_encode, path_to_save_encode_segmentation):
    
    with open(path_to_dic_encode) as f:
        dic_encode = json.load(f)
        
    segm = open_file(path_to_segmentation)
    segm = [el.split(' ') for el in segm]
    segm = [list(map(dic_encode.get, el)) for el in segm]
    segm = [' '.join(el) for el in segm]
    segm = '\n'.join(segm)
    
    write_file(path_to_save_encode_segmentation, segm)

In [ ]:
# path_to_segmentation = '/Data/models/topline_phoneme_smallbert/dpparse/construction/segmentation_dpparse.txt'
# path_to_save_dic_encode = '/Data/models/topline_phoneme_smallbert/dpparse/construction/dic_encode_dpparse.json'
# create_dic_encode_phonemes(path_to_segmentation, path_to_save_dic_encode)

# path_to_segmentation = '/Data/models/topline_phoneme_smallbert/dpparse_noduration/construction/segmentation_dpparse_noduration.txt'
# path_to_save_dic_encode = '/Data/models/topline_phoneme_smallbert/dpparse_noduration/construction/dic_encode_dpparse_noduration.json'
# create_dic_encode_phonemes(path_to_segmentation, path_to_save_dic_encode)


# path_to_segmentation = '/Data/models/topline_phoneme_smallbert/dpparse/construction/segmentation_dpparse.txt'
# path_to_dic_encode = '/Data/models/topline_phoneme_smallbert/dpparse/construction/dic_encode_dpparse.json'
# path_to_save_encode_segmentation = '/Data/models/topline_phoneme_smallbert/dpparse/construction/quantized/quantized_libri_dpparse.txt'
# create_quantized_libri(path_to_segmentation, path_to_dic_encode, path_to_save_encode_segmentation)

# path_to_segmentation = '/Data/models/topline_phoneme_smallbert/dpparse_noduration/construction/segmentation_dpparse_noduration.txt'
# path_to_dic_encode = '/Data/models/topline_phoneme_smallbert/dpparse_noduration/construction/dic_encode_dpparse_noduration.json'
# path_to_save_encode_segmentation = '/Data/models/topline_phoneme_smallbert/dpparse_noduration/construction/quantized/quantized_libri_dpparse_noduration.txt'
# create_quantized_libri(path_to_segmentation, path_to_dic_encode, path_to_save_encode_segmentation)


## Create quantized

In [ ]:
def create_quantized_task(path_to_normalseg_task, path_to_dic_encode, path_to_save_quantized):
    
    with open(path_to_dic_encode) as f:
        dic_encode = json.load(f)
        
    normalseg = open_file(path_to_normalseg_task)
    
    codes = [el.split(' ')[0] for el in normalseg]
    seg = [el.split(' ')[1:] for el in normalseg]
    
    quantized = [codes[i] + '\t' + ','.join(list(map(dic_encode.get, el))) for i, el in enumerate(seg) if all(unit in dic_encode for unit in el)]
    quantized = '\n'.join(quantized)
    
    write_file(path_to_save_quantized, quantized)

In [ ]:
# path_to_normalseg_task = '/normalseg_tasks_dpparse.txt'
# path_to_dic_encode = '/Data/models/topline_phoneme_smallbert/dpparse/construction/dic_encode_dpparse.json'
# path_to_save_quantized = '/quantized_tasks_dpparse.txt'
# create_quantized_task(path_to_normalseg_task, path_to_dic_encode, path_to_save_quantized)

# path_to_normalseg_task = '/normalseg_tasks_dpparse_noduration.txt'
# path_to_dic_encode = '/Data/models/topline_phoneme_smallbert/dpparse_noduration/construction/dic_encode_dpparse_noduration.json'
# path_to_save_quantized = '/quantized_tasks_dpparse_noduration.txt'
# create_quantized_task(path_to_normalseg_task, path_to_dic_encode, path_to_save_quantized)

# BPE to normal segmentation

In [ ]:
def bpe_to_normal_segmentation(path_to_bpe, path_save_normal_representation, for_task = False):
    
    f = open(path_to_bpe, 'r')
    txt_bpe = f.read()
    f.close()
    
    txt_bpe = [el for el in txt_bpe.split('\n') if el != '']
    
    if for_task: # we give the quantized for a task for the bpe model as input
        txt_bpe = [el.split('\t')[1].replace(',', ' ') for el in txt_bpe]
    
    normal_repres = []
    
    for line in txt_bpe:
        line_split = line.split(' ')
        line_split = list(map(int, line_split))
        
        ind_beg_bpe = [i for i, el in enumerate(line_split) if el <= 59]
        
        normal_line = []
        
        for i in range(len(ind_beg_bpe) - 1):
            bound = ind_beg_bpe[i + 1] - ind_beg_bpe[i]
            new_bpe = line_split[ind_beg_bpe[i] : ind_beg_bpe[i] + bound]
            new_bpe = [el if el <= 59 else el-59 for el in new_bpe]
            new_bpe = ','.join(list(map(str, new_bpe)))
            normal_line.append(new_bpe)
        new_bpe = line_split[ind_beg_bpe[len(ind_beg_bpe) - 1]:]
        new_bpe = [el if el <= 59 else el-59 for el in new_bpe]
        new_bpe = ','.join(list(map(str, new_bpe)))
        normal_line.append(new_bpe)
        
        normal_line = ' '.join(normal_line)
        normal_repres.append(normal_line)
        
    normal_repres = '\n'.join(normal_repres)
    
    f = open(path_save_normal_representation, 'w')
    f.truncate(0)
    f.write(normal_repres)
    f.close()

In [ ]:
# path_to_bpe = '/Data/models/topline_phoneme_smallbert/no_boundaries_masking_bpe/no_boundaries_masking_bpe_1000/construction/quantized/phonemes_noboundaries_masking_bpe_1000_preprocessed_train_text_librispeech.txt'
# path_save_normal_representation = '/normalseg_phonemes_noboundaries_masking_bpe_1000_preprocessed_train_text_librispeech.txt'
# for_task = False
# bpe_to_normal_segmentation(path_to_bpe, path_save_normal_representation, for_task)